# Modelo para detecção de fraudes

O objetivo deste projeto é desenvolver um modelo de rede neural capaz de prever fraudes no consumo de água. Essa tarefa é de extrema importância, pois fraudes podem representar perdas significativas para empresas de saneamento. Para isso, utilizamos uma combinação de técnicas avançadas de aprendizado de máquina e estratégias de otimização de hiperparâmetros. O relatório detalha as etapas tomadas, com foco nas escolhas de hiperparâmetros e o impacto que essas escolhas tiveram no desempenho final do modelo.

Este relatório está dividido em três partes principais:
- **Otimização de Hiperparâmetros**: Análise das estratégias utilizadas e impacto no desempenho.
- **Análise de Impacto**: Discussão detalhada sobre como cada hiperparâmetro influencia a performance.
- **Resultados Finais e Conclusões**: Apresentação do modelo otimizado e justificativa das escolhas.


## Instalação e download de dados

In [ ]:
import pandas as pd
import tensorflow as tf
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
import tensorflow_addons as tfa
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
# Importa o regressor de processo gaussiano, que modela a função de perda como uma distribuição probabilística
from sklearn.gaussian_process import GaussianProcessRegressor

# Importa o kernel Matern, que define como dois pontos são considerados similares em um processo gaussiano
from sklearn.gaussian_process.kernels import Matern

# Função principal da otimização bayesiana que minimiza a função objetivo de forma eficiente
from skopt import gp_minimize # type: ignore

# Importa definições para criar o espaço de busca de hiperparâmetros (números reais e inteiros)
from skopt.space import Real, Integer  # type: ignore


tf.random.set_seed(42)

In [ ]:
!gdown 1iZDFEefN0J2wao1QRIAuT-ukp0coX0Rp
!gdown 1_lY6ydxyDA9-HNrYleq4UrL-JKcKLM7c
!gdown 1C1ZHPeYF71NVVOkZD6cW5SmzV9Uui4QK
!gdown 1Lbayox3-fo92nLSvk1MPbai5ek0Noqi3

In [ ]:
df = pd.read_csv('/content/DADOS_PROCESSADOS (1).csv', delimiter=',')
df

In [ ]:
df_ext = pd.read_csv('/content/DADOS_PROCESSADOS_COMPLETOS.csv', delimiter=',')
df_ext

Os dados utilizados no modelo apresentado a seguir foram selecionados com base nos insights obtidos durante a análise exploratória. Decidimos focar em três variáveis principais: consumo, localização e categoria. Essa escolha foi motivada pelo nosso objetivo de avaliar se essas variáveis isoladas seriam suficientes para que o modelo compreendesse os padrões de consumo e comportamento dos clientes da Aegea.

Os resultados preliminares indicaram que o modelo foi capaz de alcançar uma performance razoável utilizando apenas essas três variáveis. No entanto, durante a apresentação, nosso parceiro de projeto expressou preocupação com a limitação imposta pela utilização de apenas três variáveis. Reconhecemos a validade dessa observação e, embora tenhamos decidido manter a abordagem inicial para fins comparativos, planejamos incorporar outras variáveis em etapas subsequentes para aprimorar o desenvolvimento do modelo.

In [ ]:
df = df.drop(columns=['MES_13', 'MES_14', 'MES_15',
       'MES_16', 'MES_17', 'MES_18', 'MES_19', 'MES_20', 'MES_21', 'MES_22',
       'MES_23', 'MES_24', 'MES_25', 'MES_26', 'MES_27', 'MES_28', 'MES_29',
       'MES_30', 'MES_31', 'MES_32', 'MES_33', 'MES_34', 'MES_35', 'MES_36'])

In [ ]:
df_ext = df_ext.drop(columns=['MATRICULA', 'CONS_MEDIDO', 'ANOMES', 'COD_LATITUDE', 'COD_LONGITUDE', 'CLUSTER', 'CONTAGEM_MATRICULA','MES_7', 'MES_8', 'MES_9', 'MES_10', 'MES_11', 'MES_12', 'MES_13', 'MES_14', 'MES_15', 'ECO_INDUSTRIAL', 'ECO_COMERCIAL', 'ECO_PUBLICA', 'ECO_OUTRAS',    'COD_GRUPO', 'COD_SETOR_COMERCIAL', 'COD_SETOR_COMERCIAL.1',
       'MES_16', 'MES_17', 'MES_18', 'MES_19', 'MES_20', 'MES_21', 'MES_22',
       'MES_23', 'MES_24', 'MES_25', 'MES_26', 'MES_27', 'MES_28', 'MES_29',
       'MES_30', 'MES_31', 'MES_32', 'MES_33', 'MES_34', 'MES_35', 'MES_36'])

Aqui, dividimos a coluna target das outras colunas utilizadas como features do moddelo.

In [ ]:
X = df.drop(columns=['FRAUDADOR'])
y = df['FRAUDADOR']
X

In [ ]:
X_ext = df_ext.drop(columns=['FRAUDADOR'])
y_ext = df_ext['FRAUDADOR']
X_ext

## Equilíbrio de classes por meio do método undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt

# Analisando o desequilíbrio das classes
y.value_counts().plot(kind='bar')
plt.title('Distribuição das Classes')
plt.show()

# Aplicando RandomUnderSampler para balanceamento
undersampler = RandomUnderSampler(random_state=42)
X_balanced, y_balanced = undersampler.fit_resample(X, y)

# Verificando a nova distribuição das classes
y_balanced.value_counts().plot(kind='bar')
plt.title('Distribuição das Classes Após Undersampling')
plt.show()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt

# Analisando o desequilíbrio das classes
y_ext.value_counts().plot(kind='bar')
plt.title('Distribuição das Classes')
plt.show()

# Aplicando RandomUnderSampler para balanceamento
undersampler = RandomUnderSampler(random_state=42)
X_balanced_ext, y_balanced_ext = undersampler.fit_resample(X_ext, y_ext)

# Verificando a nova distribuição das classes
y_balanced_ext.value_counts().plot(kind='bar')
plt.title('Distribuição das Classes Após Undersampling')
plt.show()

## Arquitetura e estruturação do modelo de redes neurais

O modelo de rede neural escolhido segue uma arquitetura piramidal, na qual o número de neurônios em cada camada subsequente diminui, criando um funil de informações que permite ao modelo captar padrões mais complexos nas camadas superiores e refinar os detalhes nas camadas mais profundas.

A função de ativação **ReLU (Rectified Linear Unit)** foi selecionada para as camadas ocultas devido à sua eficiência em evitar o problema de vanishing gradient, permitindo que o modelo aprenda de forma mais eficiente em problemas de classificação. A última camada usa a função **sigmoid**, que transforma a saída do modelo em uma probabilidade, apropriada para problemas binários como este.


Divisão dos dados em treino, teste e validação.

In [ ]:
# Primeiro, divisão dos dados em treino (70%) e teste (30%)
X_train_full, X_test, y_train_full, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

# Agora, dividimos o conjunto de treino em treino (70% do total) e validação (30% do total)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.3, random_state=42)

In [ ]:
# Primeiro, divisão dos dados em treino (70%) e teste (30%)
X_train_full_ext, X_test_ext, y_train_full_ext, y_test_ext = train_test_split(X_balanced_ext, y_balanced_ext, test_size=0.3, random_state=42)

# Agora, dividimos o conjunto de treino em treino (70% do total) e validação (30% do total)
X_train_ext, X_val_ext, y_train_ext, y_val_ext = train_test_split(X_train_full_ext, y_train_full_ext, test_size=0.3, random_state=42)

Desenvolvemos um modelo de rede neural utilizando uma arquitetura piramidal, onde o número de neurônios em cada camada corresponde a potências de dois. Essa escolha visa explorar a eficiência e a capacidade de generalização dessa estrutura.

A camada de saída do modelo foi configurada para gerar valores probabilísticos entre 0 e 1, o que nos levou a optar pela função de perda 'binary crossentropy'. Esta função é ideal para medir a discrepância entre as probabilidades previstas pelo modelo e as classes reais binárias, auxiliando no ajuste eficaz dos pesos durante o treinamento.

## Criação do modelo

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compilando o modelo
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# Resumo da arquitetura do modelov
model.summary()

In [ ]:
model_ext = Sequential()
model_ext.add(Dense(128, input_dim=X_train_ext.shape[1], activation='relu'))
model_ext.add(Dense(64, activation='relu'))
model_ext.add(Dense(32, activation='relu'))
model_ext.add(Dense(16, activation='relu'))
model_ext.add(Dense(8, activation='relu'))
model_ext.add(Dense(1, activation='sigmoid'))
# Compilando o modelo
model_ext.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Resumo da arquitetura do modelov
model_ext.summary()

## Compilação do modelo

In [ ]:
optimizer = RMSprop(learning_rate=0.001)

model.compile(loss='binary_crossentropy',
              optimizer='RMSprop',
              metrics=[
                  'accuracy',
                  tfa.metrics.F1Score(num_classes=1, threshold=0.5),
                  tf.keras.metrics.Precision(),
                  tf.keras.metrics.Recall()
              ])

In [ ]:
optimizer = RMSprop(learning_rate=0.0001)

model_ext.compile(loss='binary_crossentropy',
              optimizer='RMSprop',
              metrics=[
                  'accuracy',
                  tfa.metrics.F1Score(num_classes=1, threshold=0.5),
                  tf.keras.metrics.Precision(),
                  tf.keras.metrics.Recall()
              ])

## Treinamento do modelo

In [ ]:
# Definir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# class_weight = {0: 1.5, 1: 2}  # Pese mais a classe positiva (classe 1)
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping],
                    # class_weight=class_weight
                    )

# # Avaliar o modelo no conjunto de teste
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}, F1-Score: {f1_score}, Precision: {precision}, Recall: {recall}')

In [ ]:
# Definir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# class_weight = {0: 1, 1: 1.3}  # Pese mais a classe positiva (classe 1)
history_ext = model_ext.fit(X_train_ext, y_train_ext, epochs=100, batch_size=32,
                    validation_data=(X_test_ext, y_test_ext),
                    callbacks=[early_stopping],
                    # class_weight=class_weight
                    )

# # Avaliar o modelo no conjunto de teste
loss, accuracy, f1_score, precision, recall = model_ext.evaluate(X_test_ext, y_test_ext)
print(f'Loss: {loss}, Accuracy: {accuracy}, F1-Score: {f1_score}, Precision: {precision}, Recall: {recall}')

## Avaliação da performance do modelo no conjunto de validação

In [ ]:
# Avaliar o modelo no conjunto de teste
results = model.evaluate(X_test, y_test)
test_loss, test_accuracy, test_f1_score, test_precision, test_recall = results
print(f'Conjunto de Teste - Loss: {test_loss}, Accuracy: {test_accuracy}, F1-Score: {test_f1_score}, Precision: {test_precision}, Recall: {test_recall}')

# Avaliar o modelo no conjunto de validação
results_val = model.evaluate(X_val, y_val)
val_loss, val_accuracy, val_f1_score, val_precision, val_recall = results_val
print(f'Conjunto de Validação - Loss: {val_loss}, Accuracy: {val_accuracy}, F1-Score: {val_f1_score}, Precision: {val_precision}, Recall: {val_recall}')

In [ ]:
# Avaliar o modelo no conjunto de teste
results = model_ext.evaluate(X_test_ext, y_test_ext)
test_loss, test_accuracy, test_f1_score, test_precision, test_recall = results
print(f'Conjunto de Teste - Loss: {test_loss}, Accuracy: {test_accuracy}, F1-Score: {test_f1_score}, Precision: {test_precision}, Recall: {test_recall}')

# Avaliar o modelo no conjunto de validação
results_val = model_ext.evaluate(X_val_ext, y_val_ext)
val_loss, val_accuracy, val_f1_score, val_precision, val_recall = results_val
print(f'Conjunto de Validação - Loss: {val_loss}, Accuracy: {val_accuracy}, F1-Score: {val_f1_score}, Precision: {val_precision}, Recall: {val_recall}')

## Hiperparametrização

A otimização de hiperparâmetros é uma etapa crucial no desenvolvimento de modelos de rede neural, pois os valores corretos podem aumentar significativamente o desempenho do modelo. Para esse projeto, optamos por duas abordagens complementares:

1. **RandomizedSearchCV**: Explora aleatoriamente o espaço de hiperparâmetros. Essa técnica é eficiente quando o espaço de busca é grande, pois evita a exaustividade do grid search e cobre uma amostra representativa dos parâmetros.
2. **Bayesian Optimization (com skopt)**: É uma abordagem mais sofisticada, que constrói um modelo probabilístico do espaço de busca e usa esse modelo para selecionar as amostras subsequentes. Esta técnica é especialmente útil quando o custo computacional é alto, pois reduz o número de iterações necessárias para encontrar bons resultados.

Utilizamos essas duas técnicas para otimizar os seguintes hiperparâmetros:
- **Número de neurônios por camada**: Testamos entre 16 e 256 neurônios.
- **Taxa de aprendizado**: Variamos a taxa de aprendizado de 1e-5 a 1e-2.
- **Funções de ativação**: Testamos ReLU, sigmoid, tanh e softmax.
- **Taxa de dropout**: Para prevenir overfitting, testamos diferentes taxas de dropout.
- **Otimizador**: Comparando entre Adam, RMSprop, e SGD.


In [ ]:
# from skopt.space import Categorical

# # Definindo o espaço de busca
# space2 = [
#     Real(1e-5, 1e-2, name='learning_rate'),       # Espaço para a taxa de aprendizado
#     Integer(16, 258, name='num_neurons'),          # Espaço para o número de neurônios
#     Integer(16, 256, name='batch_size'),          # Espaço para o tamanho do batch
#     Integer(1, 10, name='num_layers'),             # Número de camadas
#     Categorical(['relu', 'tanh', 'sigmoid','softmax'], name='activation'),  # Funções de ativação
#     Real(0.0, 1, name='dropout_rate'),          # Taxa de dropout
#     Categorical(['adam', 'sgd', 'rmsprop'], name='optimizer')     # Otimizador
# ]

In [ ]:
# # Função de avaliação para a Otimização Bayesiana
# def train_eval(params):
#     learning_rate, num_neurons, batch_size, num_layers, activation, dropout_rate, optimizer = params

#     # Criar o modelo
#     model = create_model(learning_rate, num_neurons)

#     # Treinar o modelo
#     history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

#     # Avaliar o modelo
#     _, accuracy = model.evaluate(X_test, y_test, verbose=0)

#     # A função de perda para a otimização bayesiana precisa ser minimizada, então retornamos 1 - accuracy
#     return 1 - accuracy

In [ ]:
# # Executando a Otimização Bayesiana
# result2 = gp_minimize(train_eval, space2, n_calls=10, random_state=42)

In [ ]:
# 1# Exibir os melhores hiperparâmetros encontrados
# print(f"Melhor taxa de aprendizado: {result2.x[0]}")
# print(f"Melhor número de neurônios: {result2.x[1]}")
# print(f"Melhor resultado (1 - accuracy): {result2.fun}")
# print(f"Melhor batch size: {result2.x[2]}")
# print(f"Melhor número de camadas: {result2.x[3]}")
# print(f"Melhor função de ativação: {result2.x[4]}")
# print(f"Melhor taxa de dropout: {result2.x[5]}")
# print(f"Melhor otimizador: {result2.x[6]}")

In [ ]:
from scikeras.wrappers import KerasClassifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import pandas as pd

In [ ]:
# Função para criar o modelo Keras
def create_model(num_neurons=32, num_layers=4, activation='relu', dropout_rate=0.0, learning_rate=0.001, optimizer='adam'):
    model = Sequential()
    model.add(tf.keras.Input(shape=(X_train.shape[1],)))

    for _ in range(num_layers):
        model.add(Dense(num_neurons, activation=activation))
        if dropout_rate > 0.0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))

    # Escolher o otimizador
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Preparar o KerasClassifier com os parâmetros para o Random Search
model = KerasClassifier(model=create_model, verbose=0)

# Definir os hiperparâmetros a serem testados no Random Search
param_dist = {
    'model__learning_rate': uniform(1e-5, 1e-2),      # Amostra valores para taxa de aprendizado
    'model__num_neurons': randint(16, 256),           # Amostra valores entre 16 e 256 para o número de neurônios
    'model__num_layers': randint(1, 20),              # Amostra valores entre 1 e 10 para o número de camadas
    'model__activation': ['relu', 'tanh', 'sigmoid', 'softmax'], # Escolher entre essas funções de ativação
    'model__dropout_rate': uniform(0.0, 1),         # Amostra valores para a taxa de dropout
    'model__optimizer': ['adam', 'sgd', 'rmsprop', 'lion'],   # Escolher entre esses otimizadores
    'batch_size': randint(16, 256),                   # Amostra valores para o tamanho do batch entre 16 e 256
    'epochs': [200]                                    # Definimos um número fixo de épocas como 50
}

# Executar o Randomized Search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=50, scoring='accuracy', cv=3, verbose=2, random_state=42, n_jobs=-1)

# Treinamento e busca dos melhores parâmetros
random_search_result = random_search.fit(X_train, y_train)

# Exibir os melhores parâmetros encontrados
best_params = random_search_result.best_params_
print(f"Melhores Hiperparâmetros: {best_params}")



## Análise de impacto dos hiperparâmetros

A seguir, discutimos o impacto de cada um dos principais hiperparâmetros no desempenho do modelo:



Melhores Hiperparâmetros: {'batch_size': 79, 'epochs': 50, 'model__activation': 'relu', 'model__dropout_rate': 0.023225206359998862, 'model__learning_rate': 0.006085448519014384, 'model__num_layers': 5, 'model__num_neurons': 88, 'model__optimizer': 'rmsprop'}

- **Número de Neurônios**: Camadas com mais neurônios tendem a capturar padrões mais complexos, porém podem levar ao overfitting. Após a otimização, descobrimos que 88 neurônios por camada oferecia o melhor trade-off entre complexidade e generalização.
  
- **Taxa de Aprendizado**: Uma taxa de aprendizado muito alta pode levar o modelo a saltar para longe do ótimo global, enquanto uma taxa muito baixa pode fazer com que o treinamento seja muito lento. A taxa de 0.006 foi a que produziu o melhor desempenho, permitindo que o modelo convergisse rapidamente sem perder a capacidade de generalização.

- **Função de Ativação**: A função **ReLU** se mostrou superior em termos de performance geral, principalmente devido à sua capacidade de lidar bem com problemas de gradiente.

- **Taxa de Dropout**: Introduzimos uma taxa de dropout de 0.023 para prevenir overfitting, o que ajudou a melhorar a generalização do modelo.

- **Otimizador**: O **RMSprop** se mostrou o melhor otimizador para o problema, devido à sua capacidade de ajustar dinamicamente a taxa de aprendizado durante o treinamento.

In [ ]:
best_params

In [ ]:
import joblib

# Salvar o modelo otimizado
best_model = random_search_result.best_estimator_
joblib.dump(best_model, 'best_model.pkl')

# Fazer previsões no conjunto de teste
y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

# Avaliar o desempenho final
from sklearn.metrics import classification_report, confusion_matrix

print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(cm)

## Justificativa do modelo otimizado

Após a otimização dos hiperparâmetros, o modelo final apresentou os seguintes hiperparâmetros ótimos:
- **Número de neurônios**: 88
- **Taxa de aprendizado**: 0.006
- **Função de ativação**: ReLU
- **Taxa de dropout**: 0.023
- **Otimizador**: RMSprop

Os resultados mostraram uma melhora significativa nas métricas de desempenho, especialmente na precisão e no F1-Score, indicando que o modelo foi capaz de lidar bem com o desequilíbrio de classes e capturar os padrões relacionados às fraudes no consumo de água.

## Visualização gráfica

Gráfico de linha com as épocas ao longo do tempo

In [ ]:
epochs = list(range(1, len(history.history['loss']) + 1))
history_df = pd.DataFrame({
    'Epoch': epochs,
    'Training Loss': history.history['loss'],
    'Validation Loss': history.history['val_loss'],
    'Training Accuracy': history.history['accuracy'],
    'Validation Accuracy': history.history['val_accuracy']
})

fig_loss = px.line(history_df, x='Epoch', y=['Training Loss', 'Validation Loss'],
                   labels={'value': 'Loss', 'variable': 'Type'},
                   title='Loss during Training')
fig_loss.show()

fig_accuracy = px.line(history_df, x='Epoch', y=['Training Accuracy', 'Validation Accuracy'],
                       labels={'value': 'Accuracy', 'variable': 'Type'},
                       title='Accuracy during Training')
fig_accuracy.show()

In [ ]:
epochs = list(range(1, len(history.history['loss']) + 1))
history_df_ext = pd.DataFrame({
    'Epoch': epochs,
    'Training Loss': history.history['loss'],
    'Validation Loss': history.history['val_loss'],
    'Training Accuracy': history.history['accuracy'],
    'Validation Accuracy': history.history['val_accuracy']
})

fig_loss = px.line(history_df_ext, x='Epoch', y=['Training Loss', 'Validation Loss'],
                   labels={'value': 'Loss', 'variable': 'Type'},
                   title='Loss during Training')
fig_loss.show()

fig_accuracy = px.line(history_df_ext, x='Epoch', y=['Training Accuracy', 'Validation Accuracy'],
                       labels={'value': 'Accuracy', 'variable': 'Type'},
                       title='Accuracy during Training')
fig_accuracy.show()

Matriz de confusão

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
from sklearn.metrics import confusion_matrix

# Supondo que você já tenha treinado o modelo com o código anterior

# Fazer previsões no conjunto de teste
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

# Calcular a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Criar um DataFrame a partir da matriz de confusão para facilitar a visualização com Plotly
cm_df = pd.DataFrame(cm, index=['Classe 0', 'Classe 1'], columns=['Predito 0', 'Predito 1'])

# Plotar a matriz de confusão usando Plotly Express
fig = px.imshow(cm_df, text_auto=True, color_continuous_scale='Blues', aspect='auto')
fig.update_layout(
    title='Matriz de Confusão',
    xaxis_title='Predição',
    yaxis_title='Verdadeiro',
    coloraxis_showscale=False
)
fig.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# Fazer previsões no conjunto de teste
y_pred_prob_ext = model_ext.predict(X_test_ext)
y_pred_ext = (y_pred_prob_ext > 0.5).astype("int32")

# Calcular a matriz de confusão
cm = confusion_matrix(y_test_ext, y_pred_ext)

# Criar um DataFrame a partir da matriz de confusão para facilitar a visualização com Plotly
cm_df = pd.DataFrame(cm, index=['Classe 0', 'Classe 1'], columns=['Predito 0', 'Predito 1'])

# Plotar a matriz de confusão usando Plotly Express
fig = px.imshow(cm_df, text_auto=True, color_continuous_scale='Blues', aspect='auto')
fig.update_layout(
    title='Matriz de Confusão',
    xaxis_title='Predição',
    yaxis_title='Verdadeiro',
    coloraxis_showscale=False
)
fig.show()

A otimização de hiperparâmetros é um passo fundamental no desenvolvimento de modelos de aprendizado de máquina, especialmente em redes neurais, onde o ajuste fino de variáveis específicas pode afetar drasticamente o desempenho. Em termos gerais, hiperparâmetros são variáveis que controlam o processo de aprendizado de um modelo, mas que não são ajustados automaticamente durante o treinamento, como é o caso dos parâmetros de um modelo (ex: pesos das conexões em uma rede neural). Eles precisam ser definidos manualmente ou otimizados através de técnicas específicas.

Alguns dos hiperparâmetros mais importantes em redes neurais incluem:

Número de Neurônios em Cada Camada: Refere-se à quantidade de unidades processadoras de informação em cada camada da rede. Mais neurônios podem permitir que o modelo capture padrões mais complexos, mas podem aumentar o risco de overfitting e torná-lo mais suscetível a memorizar o conjunto de treino sem generalizar bem para novos dados.

Número de Camadas: A profundidade de uma rede neural, ou seja, quantas camadas ocultas o modelo possui, também afeta sua capacidade de aprendizado. Redes mais profundas conseguem representar padrões mais abstratos e complexos, mas também são mais difíceis de treinar e exigem mais recursos computacionais.

Taxa de Aprendizado (Learning Rate): Controla o tamanho do passo que o algoritmo de otimização dá em direção ao mínimo da função de perda durante o treinamento. Uma taxa de aprendizado muito alta pode fazer com que o modelo "pule" o ótimo global, enquanto uma taxa muito baixa pode levar a um treinamento excessivamente longo ou à convergência para um mínimo local de baixa qualidade.

Função de Ativação: Define como a informação é processada em cada neurônio. Algumas das funções mais comuns são ReLU (Rectified Linear Unit), que é amplamente utilizada devido à sua eficiência e ao fato de reduzir problemas como o vanishing gradient, além de funções como sigmoid, tanh e softmax, que têm diferentes vantagens dependendo da natureza do problema.

Taxa de Dropout: Dropout é uma técnica de regularização que desativa aleatoriamente uma fração dos neurônios durante o treinamento para prevenir overfitting. A escolha da taxa de dropout ideal garante que o modelo não dependa de neurônios específicos e, ao mesmo tempo, não descarte neurônios em excesso, o que poderia prejudicar o aprendizado.

Otimizador: O algoritmo utilizado para minimizar a função de perda. Entre os mais comuns estão Adam, RMSprop e SGD (Stochastic Gradient Descent). Cada um tem características diferentes em termos de como ajustam os pesos da rede durante o treinamento, e a escolha depende das características do problema e da natureza do dataset.

Ao trabalhar com tantos hiperparâmetros, surge a questão de como ajustá-los da maneira mais eficiente possível para obter um desempenho otimizado. Uma abordagem simples seria tentar todas as combinações possíveis de hiperparâmetros (conhecida como grid search), mas essa abordagem se torna rapidamente impraticável quando há muitas combinações possíveis, pois exige um tempo de processamento muito longo. Por isso, adotamos o Random Search como a estratégia de hiperparametrização para este projeto.

Por que Escolhemos o Random Search?
Random Search é uma abordagem de otimização que, ao contrário do grid search, não explora sistematicamente todas as combinações de hiperparâmetros. Em vez disso, ele seleciona aleatoriamente um subconjunto de combinações, e o modelo é treinado e avaliado com base nessas amostras. Existem várias razões pelas quais o Random Search é especialmente eficiente para a otimização de redes neurais em comparação com outras técnicas:

Maior Eficiência em Espaços Altamente Dimensionalizados: Redes neurais possuem um grande número de hiperparâmetros, e muitas vezes apenas algumas dessas combinações de hiperparâmetros têm um impacto significativo no desempenho do modelo. O Random Search tende a explorar mais variações desses hiperparâmetros críticos em vez de gastar tempo com combinações menos relevantes, como acontece no grid search.

Cobertura Mais Eficiente do Espaço de Busca: Enquanto o grid search tende a testar todas as combinações de forma sistemática, ele pode desperdiçar muito esforço testando combinações de hiperparâmetros que são muito similares entre si, o que não agrega muito ao desempenho do modelo. O Random Search, por outro lado, distribui as tentativas de forma mais dispersa, o que muitas vezes resulta em melhores combinações com menos iterações.

Escalabilidade: Em problemas com muitos hiperparâmetros, como o nosso caso de redes neurais, a quantidade de combinações possíveis pode crescer exponencialmente. O Random Search é escalável porque você pode limitar o número de amostras que deseja testar, economizando tempo computacional e recursos. É uma abordagem que permite encontrar bons resultados com um número significativamente menor de testes do que o grid search.

Distribuição de Hiperparâmetros: Em muitas situações, algumas distribuições de hiperparâmetros (como a taxa de aprendizado ou a taxa de dropout) têm uma importância maior em intervalos específicos. O Random Search permite testar intervalos não uniformes com mais eficiência, garantindo que mais tentativas sejam feitas em regiões de maior importância para o modelo.

Flexibilidade para Ajustes Futuros: Como o Random Search é relativamente simples de configurar e ajustar, ele oferece flexibilidade para incorporar novos hiperparâmetros ou alterar o intervalo dos parâmetros existentes com facilidade. Isso é uma grande vantagem em projetos iterativos, onde a natureza do problema pode evoluir ao longo do tempo.


## Conclusão

Esta etapa demonstra a importância de uma abordagem cuidadosa e iterativa para a construção e otimização de redes neurais em problemas críticos como a detecção de fraudes. Através da aplicação de técnicas de otimização avançadas como **RandomizedSearchCV** e de outras mais simples como a **Bayesian Optimization**, fomos capazes de melhorar, mesmo que pouco, o desempenho do modelo, tornando-o um pouco mais adequado para detectar fraudes no consumo de água com alta precisão.

As técnicas e estratégias utilizadas neste projeto não apenas otimizam o desempenho do modelo, mas também buscam garantir que ele seja robusto e capaz de generalizar para novos dados. O equilíbrio entre complexidade do modelo e prevenção de overfitting foi um dos pontos centrais desta abordagem.
